Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. 
Для парсинга использовать XPath. Структура данных должна содержать:
название источника;
наименование новости;
ссылку на новость;
дата публикации.
Сложить собранные новости в БД
Минимум один сайт, максимум - все три

In [1]:
import requests
from lxml import html
from datetime import datetime
from pprint import pprint

In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient('127.0.0.1', 27017)
db = client['news_on_mail_ru']
news_mail = db.news

In [4]:
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36'}

In [5]:
url = 'https://news.mail.ru/'

In [6]:
response = requests.get(url, headers=headers).text

In [7]:
dom = html.fromstring(response)

In [8]:
items = dom.xpath("//div[@data-module='TrackBlocks' and @class='js-module']//a[@class='list__text']/@href")

In [9]:
n = 0
for link in items:
    to_base = {}
    response_news = requests.get(link, headers=headers).text
    dom_news = html.fromstring(response_news)
    
    source = dom_news.xpath("//a[@class='link color_gray breadcrumbs__link']/span[@class='link__text']/text()")
    name = dom_news.xpath("//h1/text()")
    
    datetime_string = str(dom_news.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime"))
    datetime_string_short = datetime_string.split('T')[0][2:12]
    
    to_base['source'] = source
    to_base['datetime'] = datetime_string_short
    to_base['link'] = str(link)
    to_base['name'] = str(name)
    
    news_mail.update_one({'link': to_base['link']}, {'$set': to_base}, upsert=True)
    n += 1
    print('Запись   ', n)
    

Запись    1
Запись    2
Запись    3
Запись    4
Запись    5
Запись    6
Запись    7
Запись    8
